<a href="https://colab.research.google.com/github/shanxS/2.2/blob/master/gpt_dev_0_01_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyper parameters
batch_size = 32
block_size = 8
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
#----------------

torch.manual_seed(1337)

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(list(set(''.join(text))))
vocab_size = len(chars)
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size, (batch_size,))
  x = torch.stack( [data[i: i+block_size] for i in ix] )
  y = torch.stack( [data[i+1:i+block_size+1] for i in ix] )
  x, y = x.to(device), y.to(device)
  return x, y

@torch.no_grad()
def estimate_loss():
  out = {}
  model.eval()
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for K in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[K] = loss.item()
    out[split] = losses.mean()
  model.train()
  return out




--2023-05-10 01:40:50--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.05s   

2023-05-10 01:40:50 (20.1 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



# Bigram model

In [12]:
class BigramLangugageModel(nn.Module):

  def __init__(self):
    super().__init__()
    '''
    1st argument is size of dict
    2nd argument is size of embedding for each input token
    So the dim is vocab_size x C, where C is Channel
    '''
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, input, target = None):
    '''
    This is the output. Literally just dot product of input (B x T) and embedding table (vocab_size x channel) to get embedding
    I think it internally converts input into one hot encoding of size (B x T x vocab_size)
    B: number of inputs
    T: size of each input
    C: channel
    '''
    # print(input)
    # print(target)
    
    logits = self.token_embedding_table(input) # (B x T x C)

    if target is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      target = target.view(B*T)
      loss = F.cross_entropy(logits, target)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      logits, _ = self(idx)
      logits = logits[:, -1, :] # I dont understand why last index is predicted token

      # logits = logits.exp()
      # prob = logits / logits.sum(1, keepdims = True)
      prob = F.softmax(logits, dim=-1) # (B, C)

      idx_next = torch.multinomial(prob, num_samples=1)
      idx = torch.cat((idx, idx_next), dim =1)
    return idx

# Model training & eval

In [13]:
model = BigramLangugageModel()
m = model.to(device)

optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

In [14]:
for iter in range(max_iters):
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step:{iter} training loss:{losses['train']} val loss:{losses['val']}")

  xb, yb = get_batch('train')    
  logits, loss = model(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

step:0 training loss:4.730469226837158 val loss:4.724065780639648
step:300 training loss:2.810985803604126 val loss:2.824899911880493
step:600 training loss:2.5434393882751465 val loss:2.5682179927825928
step:900 training loss:2.4931976795196533 val loss:2.508826971054077
step:1200 training loss:2.486262321472168 val loss:2.503467559814453
step:1500 training loss:2.466465711593628 val loss:2.4920854568481445
step:1800 training loss:2.4682557582855225 val loss:2.493600845336914
step:2100 training loss:2.4696457386016846 val loss:2.4845683574676514
step:2400 training loss:2.463808298110962 val loss:2.487889051437378
step:2700 training loss:2.473775863647461 val loss:2.4910616874694824


# Generate

In [15]:
context = torch.zeros((1,1), dtype = torch.long, device = device)
print(decode(m.generate(context, max_new_tokens = 400)[0].tolist()))


od nos CAy go ghanoray t, co haringoudrou clethe k,LARof fr werar,
Is fa!


Thilemel cia h hmboomyorarifrcitheviPO, tle dst f qur'dig t cof boddo y t o ar pileas h mo wierl t,
S:
STENENEat I athe thounomy tinrent distesisanimald 3I: eliento ald, avaviconofrisist me Busarend un'soto vat s k,
SBRI he the f wendleindd t acoe ts ansu, thy ppr h.QULY:
KIIsqu pr odEd ch,
APrnes ouse bll owhored miner t 


In [ ]:
len(chars)

65